In [2]:
%pip install folium
import folium
import heapq
import random

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Introduction
10 locations in the city of Los Angeles are identified and labelled as points A to J. The points are represented as nodes in a graph. The travel time from one point to another by car, taken from Google Maps, is used as the weight of each edge in the graph.

## Map Representation
Using `folium`, a map of Los Angeles is rendered with 10 markers representing distinct locations in the city. The markers represent nodes A to J in our graph. The lines between the markers represent the edges in the graph.

In [3]:
# Display map of Los Angeles showing specific locations as nodes A to J and the connections between nodes.
m = folium.Map(location=(34.0371264,-118.2895269))

# Node coordinates
coord_a = [34.0025985,-118.500227]
coord_b = [34.0681614,-118.4043514]
coord_c = [33.9550486,-118.4217691]
coord_d = [34.0873765,-118.322163]
coord_e = [34.0375508,-118.3050501]
coord_f = [33.9848599,-118.2423065]
coord_g = [34.056473,-118.2590664]
coord_h = [34.1078559,-118.1997563]
coord_i = [33.9370552,-118.1235896]
coord_j = [34.129713,-118.1210131]

# Node A: Santa Monica Pier
folium.Marker(
    location=coord_a,
    tooltip="Santa Monica Pier",
    icon=folium.Icon(icon="fa-solid fa-a")
).add_to(m)

# Node B: Rodeo Drive Walk Of Style
folium.Marker(
    location=coord_b,
    tooltip="Rodeo Drive Walk Of Style",
    icon=folium.Icon(icon="fa-solid fa-b")
).add_to(m)

# Node C: Los Angeles International Airport
folium.Marker(
    location=coord_c,
    tooltip="Los Angeles International Airport",
    icon=folium.Icon(icon="fa-solid fa-c")
).add_to(m)

# Node D: Hollywood Palladium
folium.Marker(
    location=coord_d,
    tooltip="Hollywood Palladium",
    icon=folium.Icon(icon="fa-solid fa-d")
).add_to(m)

# Node E: University of Southern California
folium.Marker(
    location=coord_e,
    tooltip="University of Southern California",
    icon=folium.Icon(icon="fa-solid fa-e")
).add_to(m)

# Node F: Plaza Americana
folium.Marker(
    location=coord_f,
    tooltip="Plaza Americana",
    icon=folium.Icon(icon="fa-solid fa-f")
).add_to(m)

# Node G: Dodger Stadium
folium.Marker(
    location=coord_g,
    icon=folium.Icon(icon="fa-solid fa-g")
).add_to(m)

# Node H: Villa's Tacos Los Angeles
folium.Marker(
    location=coord_h,
    icon=folium.Icon(icon="fa-solid fa-h")
).add_to(m)

# Node I: Stonewood Center
folium.Marker(
    location=coord_i,
    tooltip="Stonewood Center",
    icon=folium.Icon(icon="fa-solid fa-i")
).add_to(m)

# Node J: The Huntington Japanese Garden
folium.Marker(
    location=coord_j,
    tooltip="The Huntington Japanese Garden",
    icon=folium.Icon(icon="fa-solid fa-j")
).add_to(m)

# Edges
folium.PolyLine([coord_a, coord_b]).add_to(m)
folium.PolyLine([coord_a, coord_c]).add_to(m)
folium.PolyLine([coord_b, coord_c]).add_to(m)
folium.PolyLine([coord_b, coord_d]).add_to(m)
folium.PolyLine([coord_b, coord_e]).add_to(m)
folium.PolyLine([coord_c, coord_d]).add_to(m)
folium.PolyLine([coord_c, coord_e]).add_to(m)
folium.PolyLine([coord_c, coord_f]).add_to(m)
folium.PolyLine([coord_d, coord_e]).add_to(m)
folium.PolyLine([coord_d, coord_g]).add_to(m)
folium.PolyLine([coord_d, coord_h]).add_to(m)
folium.PolyLine([coord_e, coord_f]).add_to(m)
folium.PolyLine([coord_e, coord_h]).add_to(m)
folium.PolyLine([coord_e, coord_i]).add_to(m)
folium.PolyLine([coord_f, coord_i]).add_to(m)
folium.PolyLine([coord_g, coord_h]).add_to(m)
folium.PolyLine([coord_g, coord_j]).add_to(m)
folium.PolyLine([coord_h, coord_i]).add_to(m)
folium.PolyLine([coord_h, coord_j]).add_to(m)
folium.PolyLine([coord_i, coord_j]).add_to(m)

m

## Base Case
In this scenario, the graph's edges' weights represent the average time taken to get from one point to another.

In [4]:
nodes = list("ABCDEFGHIJ")

In [5]:
# Each weighted edge is a tuple of (source, destination, drive-time in minutes)
weighted_edges = [
    ("A", "B", 18),
    ("A", "C", 20),
    ("B", "C", 24),
    ("B", "D", 20),
    ("B", "E", 22),
    ("C", "D", 30),
    ("C", "E", 22),
    ("C", "F", 26),
    ("D", "E", 16),
    ("D", "G", 14),
    ("D", "H", 16),
    ("E", "F", 14),
    ("E", "H", 16),
    ("E", "I", 24),
    ("F", "I", 24),
    ("G", "H", 22),
    ("G", "J", 22),
    ("H", "I", 24),
    ("H", "J", 18),
    ("I", "J", 35)
]

In [6]:
# Use adjacency list to represent graph
# Adjacency list is a nested dictionary
# The outer layer's keys are A to J, its values are a dictionary of each node's adjacent nodes and corresponding weights.
# The inner layer's keys are each node's adjacent nodes, its values are the corresponding weights of edges to the adjacent nodes.
adj_list = {u: {} for u in nodes}
for u, v, w in weighted_edges:
  adj_list[u][v] = w
  adj_list[v][u] = w

adj_list

{'A': {'B': 18, 'C': 20},
 'B': {'A': 18, 'C': 24, 'D': 20, 'E': 22},
 'C': {'A': 20, 'B': 24, 'D': 30, 'E': 22, 'F': 26},
 'D': {'B': 20, 'C': 30, 'E': 16, 'G': 14, 'H': 16},
 'E': {'B': 22, 'C': 22, 'D': 16, 'F': 14, 'H': 16, 'I': 24},
 'F': {'C': 26, 'E': 14, 'I': 24},
 'G': {'D': 14, 'H': 22, 'J': 22},
 'H': {'D': 16, 'E': 16, 'G': 22, 'I': 24, 'J': 18},
 'I': {'E': 24, 'F': 24, 'H': 24, 'J': 35},
 'J': {'G': 22, 'H': 18, 'I': 35}}

## Rush Hour Scenario

In this scenario, we simulated rush hour traffic conditions by multiplying the time taken for each edge by a random float value between 1 and 3.

In [7]:
# To create rushhour edges, loop through weighted_edges and multiply each time by random value between 1 and 3
weighted_edges_rushhour = []
for source, dest, time in weighted_edges:#_rushhour:
  time = time*random.uniform(1,3)
  weighted_edges_rushhour.append((source, dest, time))

# Create new adjacency list using the new edges
adj_list_rushhour = {u: {} for u in nodes}
for u, v, w in weighted_edges_rushhour:
  adj_list_rushhour[u][v] = w
  adj_list_rushhour[v][u] = w

adj_list_rushhour


{'A': {'B': 26.79998302320564, 'C': 33.60167585011861},
 'B': {'A': 26.79998302320564,
  'C': 39.08639350471932,
  'D': 45.9612350893995,
  'E': 36.08670241403615},
 'C': {'A': 33.60167585011861,
  'B': 39.08639350471932,
  'D': 76.20955772193695,
  'E': 63.51986168669515,
  'F': 56.20716830122139},
 'D': {'B': 45.9612350893995,
  'C': 76.20955772193695,
  'E': 40.56794285223924,
  'G': 20.435307584444217,
  'H': 17.589725033696794},
 'E': {'B': 36.08670241403615,
  'C': 63.51986168669515,
  'D': 40.56794285223924,
  'F': 18.462359747498162,
  'H': 42.70210687837496,
  'I': 25.683074256057424},
 'F': {'C': 56.20716830122139,
  'E': 18.462359747498162,
  'I': 48.677046799150524},
 'G': {'D': 20.435307584444217, 'H': 25.013665815904, 'J': 22.07700327852358},
 'H': {'D': 17.589725033696794,
  'E': 42.70210687837496,
  'G': 25.013665815904,
  'I': 52.42934403615982,
  'J': 29.655564269438955},
 'I': {'E': 25.683074256057424,
  'F': 48.677046799150524,
  'H': 52.42934403615982,
  'J': 68.43

## Dijkstra's Algorithm

In [8]:
# Dijkstra's algorithm

def dijkstra(adj_list, start_node):
  # Track mininum distance between start_node and each node. Initialize each minimum distance as inf. Distance from one node to itself is zero.
  min_dist = {u: float("inf") for u in adj_list}
  min_dist[start_node] = 0
  # Track parent of traversed nodes for rebuilding shortest path from start_node to target_node
  parent = {u: None for u in adj_list}
  # Traverse nodes in order of priority queue. Start with start_node
  priority_queue = [(0, start_node)]
  while priority_queue:
    current_min_dist, current_node = heapq.heappop(priority_queue)
    # Ignore if min_dist to current node is larger than existing value in min_dist
    if current_min_dist > min_dist[current_node]:
      continue
    # Find distance to each adj node = distance to current node + weight of edge to adj node
    for adj_node, weight in adj_list[current_node].items():
      new_dist = current_min_dist + weight
      # If new distance to adj node is smaller than existing value in min_dist, update min_dist
      if new_dist < min_dist[adj_node]:
        min_dist[adj_node] = new_dist
        # Log update of new min_dist to adj_node
        print(f"The minimum distance from Node {start_node} to Node {adj_node} is now {new_dist}.")
        # Save adj_node's parent to rebuild shortest path from start_node to target_node
        parent[adj_node] = current_node
        # Add adj node and updated min_dist back into priority queue to evaluate against for potentially smaller min_dist
        heapq.heappush(priority_queue, (new_dist, adj_node))
  return min_dist, parent

# Taking shortest path from Dijkstra's Algorithm, create sequence of nodes traversed from start to end
# To create sequence, add end node to path, then find and add its parent nodes from parent until the start node
def show_shortest_path(parent, end_node):
  if end_node not in parent:
    return None
  path = []
  current_node = end_node
  while current_node:
    path.append(current_node)
    current_node = parent[current_node]
  # Reverse path to show sequence from start to end
  path.reverse()
  return path


In [9]:
# Run Dijkstra's Algorithm on Los Angeles map to find shortest path from A to J
aj_min_dist, aj_parent = dijkstra(adj_list, "A")

The minimum distance from Node A to Node B is now 18.
The minimum distance from Node A to Node C is now 20.
The minimum distance from Node A to Node D is now 38.
The minimum distance from Node A to Node E is now 40.
The minimum distance from Node A to Node F is now 46.
The minimum distance from Node A to Node G is now 52.
The minimum distance from Node A to Node H is now 54.
The minimum distance from Node A to Node I is now 64.
The minimum distance from Node A to Node J is now 74.
The minimum distance from Node A to Node J is now 72.


In [10]:
# Shortest path from A to J
print(f"The shortest path from Node A to Node J is: {show_shortest_path(aj_parent, "J")} with a time of {aj_min_dist["J"]} minutes.")

The shortest path from Node A to Node J is: ['A', 'B', 'D', 'H', 'J'] with a time of 72 minutes.


In [11]:
# Run Dijkstra's Algorithm on Los Angeles map to find shortest path from J to A
ja_min_dist, ja_parent = dijkstra(adj_list, "J")

The minimum distance from Node J to Node G is now 22.
The minimum distance from Node J to Node H is now 18.
The minimum distance from Node J to Node I is now 35.
The minimum distance from Node J to Node D is now 34.
The minimum distance from Node J to Node E is now 34.
The minimum distance from Node J to Node B is now 54.
The minimum distance from Node J to Node C is now 64.
The minimum distance from Node J to Node C is now 56.
The minimum distance from Node J to Node F is now 48.
The minimum distance from Node J to Node A is now 72.


In [12]:
# Shortest path from J to A
print(f"The shortest path from Node J to Node A is: {show_shortest_path(ja_parent, "A")} with a time of {ja_min_dist["A"]} minutes.")

The shortest path from Node J to Node A is: ['J', 'H', 'D', 'B', 'A'] with a time of 72 minutes.


In [13]:
# Run Dijkstra's Algorithm on Los Angeles rush hour map to find shortest path from A to J
aj_min_dist_rh, aj_parent_rh = dijkstra(adj_list_rushhour, "A")

The minimum distance from Node A to Node B is now 26.79998302320564.
The minimum distance from Node A to Node C is now 33.60167585011861.
The minimum distance from Node A to Node D is now 72.76121811260515.
The minimum distance from Node A to Node E is now 62.88668543724179.
The minimum distance from Node A to Node F is now 89.80884415134.
The minimum distance from Node A to Node F is now 81.34904518473995.
The minimum distance from Node A to Node H is now 105.58879231561676.
The minimum distance from Node A to Node I is now 88.5697596932992.
The minimum distance from Node A to Node G is now 93.19652569704937.
The minimum distance from Node A to Node H is now 90.35094314630194.
The minimum distance from Node A to Node J is now 157.0072951270332.
The minimum distance from Node A to Node J is now 120.0065074157409.
The minimum distance from Node A to Node J is now 115.27352897557294.


In [14]:
# Shortest path from A to J during rush hour conditions
print(f"The shortest path from Node A to Node J during rush hour: {show_shortest_path(aj_parent_rh, "J")} with a time of {aj_min_dist_rh["J"]} minutes.")

The shortest path from Node A to Node J during rush hour: ['A', 'B', 'D', 'G', 'J'] with a time of 115.27352897557294 minutes.


In [15]:
# Run Dijkstra's Algorithm on Los Angeles rush hour map to find shortest path from J to A
ja_min_dist_rh, ja_parent_rh = dijkstra(adj_list_rushhour, "J")

The minimum distance from Node J to Node G is now 22.07700327852358.
The minimum distance from Node J to Node H is now 29.655564269438955.
The minimum distance from Node J to Node I is now 68.43753543373398.
The minimum distance from Node J to Node D is now 42.512310862967794.
The minimum distance from Node J to Node E is now 72.35767114781392.
The minimum distance from Node J to Node B is now 88.47354595236729.
The minimum distance from Node J to Node C is now 118.72186858490474.
The minimum distance from Node J to Node F is now 117.11458223288452.
The minimum distance from Node J to Node F is now 90.82003089531207.
The minimum distance from Node J to Node A is now 115.27352897557293.


In [16]:
# Shortest path from J to A during rush hour conditions
print(f"The shortest path from Node J to Node A during rush hour: {show_shortest_path(ja_parent_rh, "A")} with a time of {ja_min_dist_rh["A"]} minutes.")

The shortest path from Node J to Node A during rush hour: ['J', 'G', 'D', 'B', 'A'] with a time of 115.27352897557293 minutes.


# Uncertainty and Unexpected Events (dynamic solution)

In [17]:
def disruption_event(event_nodes):
  """The function returns an updated list of the unexpected events """
  graph_list = []
  for u, v, e, in event_nodes:
    prob = random.random()
    if prob <= 0.2:
        disruption_type = e *10 #for severe delays
        graph_list.append((u, v, disruption_type))
        print("There is a severe delay")
    elif 0.25 <  prob < 0.8:
          max_val = 9999999999999
          val = random.randint(15,max_val)
          disruption_type = e * val #for route blockge
          graph_list.append((u, v, disruption_type))
          print("There is a route blockage")
    else:
        graph_list.append((u, v, e)) #normal-traffic
        print("There is Normal traffic")
  return graph_list


#weighted edges used are from the base case
edges_selected = random.sample(weighted_edges, 3)
print(edges_selected)
print(disruption_event(edges_selected))


#we will substitute with ones from the rush hour here.
edges_selected_rh = random.sample(weighted_edges_rushhour, 3)
print(edges_selected_rh)
print(disruption_event(edges_selected_rh))

[('I', 'J', 35), ('H', 'J', 18), ('E', 'I', 24)]
There is Normal traffic
There is a route blockage
There is a route blockage
[('I', 'J', 35), ('H', 'J', 118399129967142), ('E', 'I', 19482440364192)]
[('G', 'J', 22.07700327852358), ('B', 'C', 39.08639350471932), ('E', 'I', 25.683074256057424)]
There is a route blockage
There is a route blockage
There is a severe delay
[('G', 'J', 126431749090497.77), ('B', 'C', 344075271918978.44), ('E', 'I', 256.8307425605742)]


# Base Case traffic with uncertainty tied in

In [18]:
# Base case adjacency list tying in potential disruption
# Run disruption code to find potential affected edge
weighted_edges_base_uncertainty = []
affected_edge = disruption_event(edges_selected)

#create a map for disrupted edges
disruption_map = {}
for source, dest, new_time in affected_edge:
  affected = (source,dest)
  disruption_map[affected] = new_time

# Looping through edge list, applying any disruption
for source, dest, time in weighted_edges:
  edge = (source, dest)
  # Check if this edge is in the disruption map
  if edge in disruption_map:
    new_time = disruption_map[edge]
    weighted_edges_base_uncertainty.append((source, dest, new_time))
  # If this edge was not selected for disruption
  else:
    weighted_edges_base_uncertainty.append((source, dest, time))


# Create new adjacency list using the new edges
adj_list_base_uncertainty = {u: {} for u in nodes}
for u, v, w in weighted_edges_base_uncertainty:
  adj_list_base_uncertainty[u][v] = w
  adj_list_base_uncertainty[v][u] = w

print(edges_selected)
adj_list_base_uncertainty

There is a route blockage
There is a severe delay
There is a route blockage
[('I', 'J', 35), ('H', 'J', 18), ('E', 'I', 24)]


{'A': {'B': 18, 'C': 20},
 'B': {'A': 18, 'C': 24, 'D': 20, 'E': 22},
 'C': {'A': 20, 'B': 24, 'D': 30, 'E': 22, 'F': 26},
 'D': {'B': 20, 'C': 30, 'E': 16, 'G': 14, 'H': 16},
 'E': {'B': 22, 'C': 22, 'D': 16, 'F': 14, 'H': 16, 'I': 177037674249408},
 'F': {'C': 26, 'E': 14, 'I': 24},
 'G': {'D': 14, 'H': 22, 'J': 22},
 'H': {'D': 16, 'E': 16, 'G': 22, 'I': 24, 'J': 180},
 'I': {'E': 177037674249408, 'F': 24, 'H': 24, 'J': 301500576454490},
 'J': {'G': 22, 'H': 180, 'I': 301500576454490}}

In [19]:
# Run Dijkstra's Algorithm on Los Angeles base case uncertainty map to find shortest path from A to J
aj_min_dist_base, aj_parent_base = dijkstra(adj_list_base_uncertainty, "A")

The minimum distance from Node A to Node B is now 18.
The minimum distance from Node A to Node C is now 20.
The minimum distance from Node A to Node D is now 38.
The minimum distance from Node A to Node E is now 40.
The minimum distance from Node A to Node F is now 46.
The minimum distance from Node A to Node G is now 52.
The minimum distance from Node A to Node H is now 54.
The minimum distance from Node A to Node I is now 177037674249448.
The minimum distance from Node A to Node I is now 70.
The minimum distance from Node A to Node J is now 74.


In [20]:
# Shortest path from A to J during base case conditions with uncertainty tied in
print(f"The shortest path from Node A to Node J during base hour with uncertainties: {show_shortest_path(aj_parent_base, "J")} with a time of {aj_min_dist_base["J"]} minutes.")

The shortest path from Node A to Node J during base hour with uncertainties: ['A', 'B', 'D', 'G', 'J'] with a time of 74 minutes.


## Rush hour traffic with uncertainty tied in

In [21]:
# Base case adjacency list tying in potential disruption
# Run disruption code to find potential affected edge
edges_selected_rh2 = random.sample(weighted_edges_rushhour, 3)
weighted_edges_rushhour_uncertainty = []
affected_edge_rushhour = disruption_event(edges_selected_rh2)

#create a map for disrupted edges
disruption_map_rushhour = {}
for source, dest, new_time in affected_edge_rushhour:
  affected_rushhour = (source,dest)
  disruption_map_rushhour[affected_rushhour] = new_time

# Looping through edge list, applying any disruption
for source, dest, time in weighted_edges_rushhour:
  edge = (source, dest)
  # Check if this edge is in the disruption map
  if edge in disruption_map_rushhour:
    new_time = disruption_map_rushhour[edge]
    weighted_edges_rushhour_uncertainty.append((source, dest, new_time))
  # If this edge was not selected for disruption
  else:
    weighted_edges_rushhour_uncertainty.append((source, dest, time))


# Create new adjacency list using the new edges
adj_list_rushhour_uncertainty = {u: {} for u in nodes}
for u, v, w in weighted_edges_rushhour_uncertainty:
  adj_list_rushhour_uncertainty[u][v] = w
  adj_list_rushhour_uncertainty[v][u] = w

print(edges_selected_rh2)
adj_list_rushhour_uncertainty

There is a severe delay
There is Normal traffic
There is Normal traffic
[('E', 'I', 25.683074256057424), ('H', 'J', 29.655564269438955), ('C', 'D', 76.20955772193695)]


{'A': {'B': 26.79998302320564, 'C': 33.60167585011861},
 'B': {'A': 26.79998302320564,
  'C': 39.08639350471932,
  'D': 45.9612350893995,
  'E': 36.08670241403615},
 'C': {'A': 33.60167585011861,
  'B': 39.08639350471932,
  'D': 76.20955772193695,
  'E': 63.51986168669515,
  'F': 56.20716830122139},
 'D': {'B': 45.9612350893995,
  'C': 76.20955772193695,
  'E': 40.56794285223924,
  'G': 20.435307584444217,
  'H': 17.589725033696794},
 'E': {'B': 36.08670241403615,
  'C': 63.51986168669515,
  'D': 40.56794285223924,
  'F': 18.462359747498162,
  'H': 42.70210687837496,
  'I': 256.8307425605742},
 'F': {'C': 56.20716830122139,
  'E': 18.462359747498162,
  'I': 48.677046799150524},
 'G': {'D': 20.435307584444217, 'H': 25.013665815904, 'J': 22.07700327852358},
 'H': {'D': 17.589725033696794,
  'E': 42.70210687837496,
  'G': 25.013665815904,
  'I': 52.42934403615982,
  'J': 29.655564269438955},
 'I': {'E': 256.8307425605742,
  'F': 48.677046799150524,
  'H': 52.42934403615982,
  'J': 68.4375

In [22]:
# Run Dijkstra's Algorithm on Los Angeles rush hour uncertainty map to find shortest path from A to J
aj_min_dist_rhu, aj_parent_rhu = dijkstra(adj_list_rushhour_uncertainty, "A")

The minimum distance from Node A to Node B is now 26.79998302320564.
The minimum distance from Node A to Node C is now 33.60167585011861.
The minimum distance from Node A to Node D is now 72.76121811260515.
The minimum distance from Node A to Node E is now 62.88668543724179.
The minimum distance from Node A to Node F is now 89.80884415134.
The minimum distance from Node A to Node F is now 81.34904518473995.
The minimum distance from Node A to Node H is now 105.58879231561676.
The minimum distance from Node A to Node I is now 319.717427997816.
The minimum distance from Node A to Node G is now 93.19652569704937.
The minimum distance from Node A to Node H is now 90.35094314630194.
The minimum distance from Node A to Node I is now 130.02609198389047.
The minimum distance from Node A to Node J is now 120.0065074157409.
The minimum distance from Node A to Node J is now 115.27352897557294.


In [23]:
# Shortest path from A to J during rush hour conditions with uncertainty tied in
print(f"The shortest path from Node A to Node J during rush hour with uncertainties: {show_shortest_path(aj_parent_rhu, "J")} with a time of {aj_min_dist_rhu["J"]} minutes.")

The shortest path from Node A to Node J during rush hour with uncertainties: ['A', 'B', 'D', 'G', 'J'] with a time of 115.27352897557294 minutes.


## Discussion and Analysis
**1. How routes change**

In this assignment, we see that the fastest route changes depending on factors such as rush hour and unexpected events (route blockage, severe delay).

Here is how rush hour impacts the calculation of the shortest route:
* For the normal traffic in Los Angeles, the shortest route from A to J is
A -> B -> D -> H -> J, for a total of 72 minutes.

* However, for the case of rush hour, when each edge is multiplied randomly by 1 to 3 (simulating heavier traffic), the shortest route changes to
A -> B -> D -> G -> J with a time of ~109 minutes. If we stick to the same route, we will have added an extra 5 minutes.

* *(note that each time we run we get a different result, so this talks about a representative case)*

**2. Impact of uncertainty**
* Assumption made: We choose randomly 3 edges to experience these uncertainties. For each edge, we calculate the probability that an unexpected event happens:

  * If probability < 0.2: the edge has severe delays. The time to travel will be 10 times longer.
  * If probability is between 0.25 and 0.8: the edge hasa route blockage. The time to travel will be from 15 to infinity longer.
  * If probability is > 0.8: the edge has normal traffic.

Based on these this assumption, we calculate the route change based on unexpected events.
* For the normal traffic in Los Angeles, the shortest route from A to J is
A -> B -> D -> H -> J, for a total of 72 minutes.
  * With route blockage randomly in edge A-B, I-J, and H-I, the shortest route from A to J is now A -> C -> E -> H -> J with a time of 76 minutes, avoiding the block on edge A-B
* For the rush traffic in Los Angeles, the shortest route from A to J is A -> C -> E -> H -> J with a time of ~119 minutes.
  * With route blockage randomly in edge H-J and B-E, the shortest path from Node A to Node J during rush hour with uncertainties is A -> C -> E -> I -> J with a time of ~148 minutes, avoiding route blockage on edge H-J.
* *(note that each time we run we get a different result, so this talks about a representative case)*

**3. Interpretation of results**
* In both base case and rush hour case, the algorithm successfully finds the shortest route from A to J.
* In the case of uncertainties, the algorithm successfully identifies the affected edges, and updated the times. From these updated time, the algorithm generated a new shortest route from A to J.
* This is interesting to see as it imitates the behavior of map behavior to calculate the shortest distance from origin to destination, taking consideration of traffic condition and road blockage.

**4. Assumptions and limitations**
* We made 2 assumptions on how to calculate the unexpected events:
  * Firstly, we assume that at any given time, out of the 10 nodes, up to 3 may experience unexpected events. In other words, no more than 3 nodes will face disruptions at once.
  * Secondly, we assume how these events happen. We assign each event (normal traffic, route blockage, severe delay) based on the random probability (see more above).

These assumptions influenced how we designed the code logic.

